In [23]:
import numpy as np

n = 10
m = 50


In [24]:
# Properties:
# - Symmetric: C[i,j] = C[j,i]
# - Non-negative: C[i,j] >= 0
# - Zero diagonal: C[i,i] = 0
# - Triangle inequality (for proper metric)

# Generate fake cosine distance matrix
def make_fake_cost_matrix(n, seed=42):
    rng = np.random.RandomState(seed)
    
    # Generate random vectors
    vectors = rng.randn(n, 128)
    
    # Normalize to unit length
    vectors = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    
    # Cosine similarity -> cosine distance
    cos_sim = vectors @ vectors.T
    cos_dist = 1 - cos_sim
    
    # Ensure exact symmetry and zero diagonal
    cos_dist = (cos_dist + cos_dist.T) / 2
    np.fill_diagonal(cos_dist, 0)
    
    # Ensure non-negative (handle numerical errors)
    cos_dist = np.maximum(cos_dist, 0)
    
    return cos_dist

C_source = make_fake_cost_matrix(n)


In [25]:
# Same properties as C_source
C_target = make_fake_cost_matrix(m)

# OR generate actual simplex samples
def make_simplex_cost_matrix(m, K, seed=42):
    rng = np.random.RandomState(seed)
    
    # Sample from K-simplex using Dirichlet
    simplex_samples = rng.dirichlet(np.ones(K + 1), size=m)
    
    # Cosine distance between simplex samples
    norms = np.linalg.norm(simplex_samples, axis=1, keepdims=True)
    normalized = simplex_samples / (norms + 1e-10)
    
    cos_sim = normalized @ normalized.T
    cos_dist = 1 - cos_sim
    
    # Ensure properties
    cos_dist = (cos_dist + cos_dist.T) / 2
    np.fill_diagonal(cos_dist, 0)
    cos_dist = np.maximum(cos_dist, 0)
    
    return cos_dist

C_target = make_simplex_cost_matrix(m, K=3)

In [26]:

p = np.ones(n) / n      # uniform source distribution
q = np.ones(m) / m      # uniform target distribution

def make_fake_coupling(n, m, seed=42):
    """
    Generate a fake coupling matrix that satisfies OT constraints.
    """
    rng = np.random.RandomState(seed)
    
    # Generate random positive matrix
    T = rng.rand(n, m)
    
    # Make it satisfy marginal constraints using Sinkhorn-like normalization
    p = np.ones(n) / n
    q = np.ones(m) / m
    
    # Iterative balancing (simplified Sinkhorn)
    for _ in range(100):
        # Normalize rows
        T = T * (p / (T.sum(axis=1) + 1e-10))[:, None]
        # Normalize columns
        T = T * (q / (T.sum(axis=0) + 1e-10))[None, :]
    
    return T

T = make_fake_coupling(n, m)

# Verify properties
print(f"Row sums (should be ~{1/n:.6f}):", T.sum(axis=1)[:5])
print(f"Column sums (should be ~{1/m:.6f}):", T.sum(axis=0)[:5])
print(f"Total sum (should be ~1.0):", T.sum())
print(f"All non-negative:", (T >= 0).all())

Row sums (should be ~0.100000): [0.1 0.1 0.1 0.1 0.1]
Column sums (should be ~0.020000): [0.02 0.02 0.02 0.02 0.02]
Total sum (should be ~1.0): 0.9999999949999999
All non-negative: True


In [27]:
point_distortions_slow = np.zeros(n)

for i in range(n):
    for k in range(n):
        for j in range(m):
            for l in range(m):
                distortion_term = (C_source[i, k] - C_target[j, l]) ** 2
                point_distortions_slow[i] += T[i, j] * T[k, l] * distortion_term

# Normalize by point mass
point_distortions_slow = point_distortions_slow * (p + 1e-9) if p is not None else point_distortions_slow


In [ ]:
C_s_squared = C_source ** 2
C_t_squared = C_target ** 2

T_row_sums = T.sum(axis=1)


## Term 1
term_1 = T_row_sums * (C_s_squared @ T_row_sums)
## Term 2

term_2 = np.sum(C_source * (T @ C_target @ T.T), axis=1)

## Term 3
term_3 = (T @ C_t_squared @ T.T).sum(axis=1)

point_distortions_fast = term_1 - 2 * term_2 + term_3
point_distortions_fast = point_distortions_fast * (p + 1e-9) if p is not None else point_distortions_fast


In [29]:
diff = point_distortions_slow - point_distortions_fast
five_num_summary = np.percentile(diff, [0, 25, 50, 75, 100])
mean_diff = np.mean(diff)
std_diff = np.std(diff)

print("Five number summary of (slow - fast):")
print(f"Min: {five_num_summary[0]}")
print(f"Q1: {five_num_summary[1]}")
print(f"Median: {five_num_summary[2]}")
print(f"Q3: {five_num_summary[3]}")
print(f"Max: {five_num_summary[4]}")
print(f"Mean: {mean_diff}")
print(f"Standard deviation: {std_diff}")



Five number summary of (slow - fast):
Min: -3.0357660829594124e-17
Q1: -2.168404344971009e-18
Median: 2.6020852139652106e-18
Q3: 1.5612511283791264e-17
Max: 1.734723475976807e-17
Mean: 1.8214596497756475e-18
Standard deviation: 1.4818691895014592e-17
